# Recruit Restaurant Visitor Forecasting
表題のコンペのコードv2<br>
処理時間およそ70min<br>
* 特徴量を作成(日付、店舗のピーク曜日、座標、カテゴリ)
* カテゴリ変数を変換(店舗IDと店舗カテゴリ)
* ラグ特徴量(7,14,21日前の平均と、直前の同じ曜日の来店者数)
<br><br>
【モデル学習方法】<br>
* 4/23のvisitorsを予測
* 4/23のvisitors数を踏まえた特徴量を生成する　←ラグ特徴量を作るのにループさせているので時間がかかる
* 4/24のvisitorsを予測
* 4/24のvisitors数を踏まえた特徴量を生成する
<br>
https://www.kaggle.com/c/recruit-restaurant-visitor-forecasting

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# air_visit_data = pd.read_csv("../input/recruit-restaurant-visitor-forecasting-data/air_visit_data.csv")
# weatherData = pd.read_csv("../input/recruit-restaurant-visitor-forecasting-data/WeatherData.csv")
# air_reserve = pd.read_csv("../input/recruit-restaurant-visitor-forecasting-data/air_reserve.csv")
# air_store_info = pd.read_csv("../input/recruit-restaurant-visitor-forecasting-data/air_store_info.csv")
# area_name_mapping = pd.read_csv("../input/recruit-restaurant-visitor-forecasting-data/area_name_mapping.csv")
# date_info = pd.read_csv("../input/recruit-restaurant-visitor-forecasting-data/date_info.csv")
# hpg_reserve = pd.read_csv("../input/recruit-restaurant-visitor-forecasting-data/hpg_reserve.csv")
# hpg_store_info = pd.read_csv("../input/recruit-restaurant-visitor-forecasting-data/hpg_store_info.csv")
# store_id_relation = pd.read_csv("../input/recruit-restaurant-visitor-forecasting-data/store_id_relation.csv")

#test = pd.read_csv("../input/recruit-restaurant-visitor-forecasting-data/test.csv")

# sample_submit = pd.read_csv("../input/recruit-restaurant-visitor-forecasting-data/sample_submission.csv")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# データの読み込み
天気や日付の情報も入れているが、使っていない

In [ ]:
train = pd.read_csv("../input/recruit-restaurant-visitor-forecasting-data/air_visit_data.csv")
weatherData = pd.read_csv("../input/recruit-restaurant-visitor-forecasting-data/WeatherData.csv")
date_info = pd.read_csv("../input/recruit-restaurant-visitor-forecasting-data/date_info.csv")

# date_infoを確認

In [ ]:
date_info = date_info.rename(columns={'calendar_date': "visit_date"})
date_info

# 曜日と祝日情報をマージする
trainのvisit_dateについて、date_infoの情報を参照して結合する

In [ ]:
train = train.merge(date_info, on="visit_date",how="left")
# 曜日はあとで別の方法で加えるので、一旦削除
train = train.drop(columns="day_of_week")
train

# trainセットとわかるようにタグ付けする

In [ ]:
train["set"] = "train"
train

# trainデータを、目的変数と説明変数に分ける

In [ ]:
train_x = train.drop(columns = "visitors")
train_y = pd.DataFrame()
train_y["visitors"] = train["visitors"]

# train_yが200以上のものは外れ値として来店者数を0
本当は、その店舗の平均来店者数にすべきなのかもしれない

In [ ]:
# train_yのvisitorsが200を超える場合は0にする
train_y.loc[train_y["visitors"] > 200, "visitors"] = 0
# visitorsでソートして表示
train_y.sort_values(by='visitors', ascending=False)

In [ ]:
train_y

# 日付を切り分ける
毎月〇日は安い、とか、〇月はセール！とかありそうなので

In [ ]:
# 2017-04-18	
def back_year(txt):
    visit_date = txt
    txt_split = txt.rsplit("-",2)
    return txt_split[0]

def back_month(txt):
    visit_date = txt
    txt_split = txt.rsplit("-",2)
    #04月のような記述にならないように、int型で返す
    return int(txt_split[1])

def back_day(txt):
    visit_date = txt
    txt_split = txt.rsplit("-",2)
    return int(txt_split[2])

# year,month,dayの列を作成

In [ ]:
# year列を新規に作成する。visit_date列の値をback_year関数に入れて処理する
train_x.loc[:,"year"] = train_x["visit_date"].apply(back_year)
train_x.loc[:,"month"] = train_x["visit_date"].apply(back_month)
train_x.loc[:,"day"] = train_x["visit_date"].apply(back_day)
train_x

# 曜日を取得する
一番効くと思っている

In [ ]:
import datetime

In [ ]:
#月火水木金土日→[0,1,2,3,4,5,6]で返す
dt = datetime.datetime(2021, 4, 26)
print(dt.weekday())

In [ ]:
#月火水木金土日→[0,1,2,3,4,5,6]で返す
def back_day_of_the_week(txt):
    visit_date = txt
    txt_split = txt.rsplit("-",2)
    dt = datetime.datetime(int(txt_split[0]), int(txt_split[1]), int(txt_split[2]))
    return dt.weekday()

# 曜日の列を追加

In [ ]:
train_x.loc[:,"day_of_the_week"] = train_x["visit_date"].apply(back_day_of_the_week)
train_x

# visit_dateは削除
他のday,year,monthで代替できるので

In [ ]:
train_x = train_x.drop(columns='visit_date')

# sample submissionを読み込み、testデータに使う

In [ ]:
test = pd.read_csv("../input/recruit-restaurant-visitor-forecasting-data/sample_submission.csv")
test["set"] = "test"

In [ ]:
test_x = test.drop(columns="visitors")
test_y = pd.DataFrame()
test_y["visitors"] = test["visitors"]
# test_yの来店者数は全て-1で埋める(→ラグ特徴量計算でマイナスを平均に加えるのは不適切と考えやめた)
# test_y["visitors"] = -1
test_y

In [ ]:
print(test_x.id[0])

# test_xのidを切り分ける
test_x(sample_submit)のidが、店舗名と日付で別々になっているので、<br>
train_x(air_visit_data)と同じ形式(air_store_idとvisit_date)になるように<br>
切り分けたい

In [ ]:
# アンダーバー区切りで、右から1つのアンダーバーを分けたpandasデータフレームを返す
txt = test_x.id[0]
print(txt)
txt_split = txt.rsplit("_",1)
print(txt_split)
print(txt_split[0])
print(txt_split[1])
test_x.date_of_customer = txt_split

上記のように書けば切り分けられることがわかった

# 切り分ける関数を定義する
idを引数にして、分割後の店舗名と日付を返す関数をそれぞれつくる

In [ ]:
def back_store_name(txt):
    id = txt
    #_区切りで右から1番目の文字を切り分ける
    txt_split = txt.rsplit("_",1)
    # 切り分けた文字列のうち、0番目を返す
    return txt_split[0]

In [ ]:
def back_date_name(txt):
    id = txt
    txt_split = txt.rsplit("_",1)
    return txt_split[1]

# 新しく作った変数(air_store_idとvisit_date)を、test_xに追加する

In [ ]:
#新しい列の名前 = 引数として関数に入れて処理する列の名前
test_x.loc[:,"air_store_id"] = test_x["id"].apply(back_store_name)
test_x.loc[:,"visit_date"] = test_x["id"].apply(back_date_name)
test_x

# 祝日フラグを追加

In [ ]:
test_x = test_x.merge(date_info, on="visit_date",how="left")
# 曜日はあとで別の方法で加えるので、一旦削除
test_x = test_x.drop(columns="day_of_week")
train

# 日付情報をYear,Month,Dayに変更

In [ ]:
# year列を新規に作成する。visit_date列の値をback_year関数に入れて処理する
test_x.loc[:,"year"] = test_x["visit_date"].apply(back_year)
test_x.loc[:,"month"] = test_x["visit_date"].apply(back_month)
test_x.loc[:,"day"] = test_x["visit_date"].apply(back_day)
test_x

# 日付情報から曜日を取得

In [ ]:
test_x.loc[:,"day_of_the_week"] = test_x["visit_date"].apply(back_day_of_the_week)
test_x

# test_xのGWを休日(土曜)に変更
test_xに含まれるGW(5/3-5/5)の曜日を変更する<br>
条件によりday_of_the_weekを書き換える<br>
当初日曜日にしていたが、土曜日の方がave_visitorsが多いので土曜に変えた

In [ ]:
# test_dataの5/3-5/5までを日曜日に変更(日曜日=6)
test_x.loc[(test_x['month'] == 5) & (test_x['day'] == 3), 'day_of_the_week'] = 5
test_x.loc[(test_x['month'] == 5) & (test_x['day'] == 4), 'day_of_the_week'] = 5
test_x.loc[(test_x['month'] == 5) & (test_x['day'] == 5), 'day_of_the_week'] = 5

In [ ]:
test_x

# idとvisit_date行は削除
idは不要。これで、test_xとtrain_xが同じ形式で取得できた<br>
visit_dateも、month,day,yearで代替しているので削除

In [ ]:
test_x = test_x.drop(columns='id')
test_x = test_x.drop(columns='visit_date')

# カテゴリ変数の変換
カテゴリ変数をLabelEncoderで実施するために、<br>
全てのカテゴリ変数(ここだと店舗名)を取得する<br>
# まず、air_store_idだけをもつDataFrameを定義する

In [ ]:
train_le = pd.DataFrame()
train_le["air_store_id"] = train_x.air_store_id
test_le = pd.DataFrame()
test_le["air_store_id"] = test_x.air_store_id

In [ ]:
test_le

# 2つ(train_leとtest_le)を合体する

In [ ]:
merge_data_le = pd.merge(train_le,test_le, how="outer")
# merge_data = merge_data["air_store_id"]
# train_le = train_x("air_store_id")
# test_le = test_x("air_store_id")

In [ ]:
merge_data_le

# ラベルエンコーディングをかける
train_all_leというDataFrameにラベルエンコーディングをかける。<br>
air_store_idにラベルエンコーディングがかけられて数値に変換される<br>

In [ ]:
from sklearn.preprocessing import LabelEncoder #Library for LabelEncoding

for c in merge_data_le:
    le = LabelEncoder()
    le.fit(merge_data_le[c])
    train_le[c] = le.transform(train_le[c])
    test_le[c]= le.transform(test_le[c])

# ラベルエンコーディングしたDataFrameをtrain_x,test_xに結合
先に、エンコーディング前のair_store_idを削除してから、結合する<br>
エンコ前後の対応表を作りたいので、train_x_encに保存しておく

In [ ]:
# エンコード前後の対応確認用、本筋とは関係ない
train_x_enc = train_x.join(train_le, lsuffix='_enc')
test_x_enc = test_x.join(test_le, lsuffix='_enc')

# エンコードしたものを入れる
train_x = train_x.drop(columns = "air_store_id")
train_x = train_x.join(train_le)
test_x = test_x.drop(columns = "air_store_id")
test_x = test_x.join(test_le)

In [ ]:
train_x

In [ ]:
test_x

# データ型を確認
どこかで、train_x、train_yともに<br>
yearがint64でなくなってしまっている

In [ ]:
test_x

In [ ]:
test_x.year.dtype

# yearのデータ型をint64に変更
xgboostで読める型に変更する

In [ ]:
train_x["year"] = train_x.year.astype("int64")
test_x["year"] = test_x.year.astype("int64")

# train_xとtrain_yを結合してtrain_allにする

In [ ]:
train_all = train_x.join(train_y)
train_all

# test_yとtest_xを結合してtest_allにする

In [ ]:
test_all = test_x.join(test_y)
test_all

# train_allとtest_allをマージしmerge_dataとする
これで、trainとtestが同じ形式で得られた。
以降は、train_allとtest_allをマージしてmerge_dataにして特徴量を追加し、<br>
最終的に切り分ける事にする

In [ ]:
merge_data = pd.concat([train_all,test_all])
merge_data

# EDA
店舗IDごとにデータがいくつあるのか、日付が何種類あるのか<br>
trainに含まれるのは829店舗で、各店舗には100～300日ぶんのデータがある<br>
testに含まれるのは821店舗で、各店舗には39日ぶんのデータがある<br>

In [ ]:
print(train_x)
# 店IDごとにデータがいくつあるか確認している
print(train_x.groupby("air_store_id").count()["day"])
# testデータについても同様
print(test_x.groupby("air_store_id").count()["day"])

# 曜日ごとの来店者数を確認する
平均は、平日17-19人、金曜日23人、土曜日26人、日曜日23人<br>
最大来店者数が800人超えているデータもあるので、外れ値は無くした方がいいかも

In [ ]:
# train_xとtrain_yを結合する
train_all = train_x.join(train_y)
# 曜日ごとに、来店者数の合計、データの数、max、minを出力する
week_visitors = train_all.groupby(['day_of_the_week']).visitors.agg([sum,len,max,min])
week_visitors["ave"] = week_visitors["sum"] // week_visitors["len"]
week_visitors

In [ ]:
train_all

# 店舗ごとの平均来店者数
平均115人来店する店舗(id71番)が一番多い

In [ ]:
store_visitors = train_all.groupby(['air_store_id']).visitors.agg([sum,len,max,min])
store_visitors["ave"] = store_visitors["sum"] // store_visitors["len"]
# 来店者数の平均でソート
store_visitors.sort_values(by='ave', ascending=False)

In [ ]:
store_visitors.sort_values(by='max', ascending=False)

In [ ]:
sns.scatterplot(x=store_visitors['ave'], y=store_visitors['max'])

In [ ]:
sns.scatterplot(x=store_visitors['ave'], y=store_visitors['max'], hue=store_visitors['len'])

# 直近4月のデータを眺める
たぶん、4月が一番傾向的に近いはず。<br>
店舗ID = 7 の店の４月のデータを見てみる<br>
(ちなみに、4月のデータのない店舗もある)

In [ ]:
train_all

In [ ]:
id_0 = train_all.loc[train_all.air_store_id == 3]
id_0 = id_0.loc[id_0.month == 4]
# id_0 = id_0.loc[id_0.day == 2]
id_0_data = pd.DataFrame()
id_0_data["visitors"] = id_0.visitors
id_0_data["month"] = id_0.month
id_0_data["day"] = id_0.day
id_0_data["day_of_the_week"] = id_0.day_of_the_week
id_0_data = id_0_data.reset_index(drop=True)
id_0_data

In [ ]:
plt.figure(figsize=(10,6))
plt.title("id=2 store visitors")
sns.barplot(x=id_0_data.day, y=id_0_data['visitors'])
plt.ylabel("visitors")

# 曜日の影響度
月の来店者数の合計、平均、max,minを曜日ごとに算出

In [ ]:
id_0_data

In [ ]:
id_0_data_ana = id_0_data.groupby(['day_of_the_week']).visitors.agg([sum,len,max,min])
id_0_data_ana["ave"] = id_0_data_ana["sum"] // id_0_data_ana["len"]
id_0_data_ana

# 最もvisitorsが多い列のindexを返す

In [ ]:
max_ave_index = id_0_data_ana['ave'].idxmax()
max_ave_index

In [ ]:
train_all.loc[train_all.air_store_id == 663]

# 店舗ごとに、trainデータから特徴量を作成
* その店舗の平均来店者数
* 各曜日の来店者の平均(日曜日～土曜部)
* 祝日の来店者数の平均
* 店ごとの来店者数のmax
* 店ごとの来店者数のmin

In [ ]:
%%time
columns =["air_store_id","v_ave_all","v_ave_0", "v_ave_1","v_ave_2","v_ave_3","v_ave_4", "v_ave_5","v_ave_6","v_ave_holiday","max_visitors","min_visitors"]
v_ave =pd.DataFrame(columns=columns)

#2017/3-4月のみの平均値をとる
train_2017 = train_all.loc[(train_all.year == 2017)]
train_cho = train_2017.loc[(train_2017.month == 1) |(train_2017.month == 2) | (train_2017.month == 3)]

for c in range(0, 829,1):
    

    
    data_ave = train_cho.loc[train_cho.air_store_id == c]
    data_ave_0 = data_ave.loc[train_all.day_of_the_week == 0] #店舗IDの日曜日の来店者数のDataFrame
    data_ave_1 = data_ave.loc[train_all.day_of_the_week == 1] #店舗IDの月曜日の来店者数のDataFrame
    data_ave_2 = data_ave.loc[train_all.day_of_the_week == 2]
    data_ave_3 = data_ave.loc[train_all.day_of_the_week == 3]
    data_ave_4 = data_ave.loc[train_all.day_of_the_week == 4]
    data_ave_5 = data_ave.loc[train_all.day_of_the_week == 5]
    data_ave_6 = data_ave.loc[train_all.day_of_the_week == 6] #店舗IDの土曜日の来店者数のDataFrame
    
    
    data_ave_h = data_ave.loc[train_all.holiday_flg == 1] #祝日の来店者数

    v_ave_all = data_ave.visitors.mean() #店舗そのものの平均来店者数
    v_ave_0 =  data_ave_0.visitors.mean() #日曜日の平均
    v_ave_1 =  data_ave_1.visitors.mean()
    v_ave_2 =  data_ave_2.visitors.mean()
    v_ave_3 =  data_ave_3.visitors.mean()
    v_ave_4 =  data_ave_4.visitors.mean()
    v_ave_5 =  data_ave_5.visitors.mean()
    v_ave_6 =  data_ave_6.visitors.mean()
    
    
    
    v_ave_h =  data_ave_h.visitors.mean() #祝日の来店者数の平均
    max_visitors =  data_ave.visitors.max()
    min_visitors =  data_ave.visitors.min()
    
    # 例外
    # 以下の店舗は欠損するので0で埋める
    if c == 110:
        v_ave_all = 0
        max_visitors = 0
        min_visitors = 0
        
    if c == 663:
        v_ave_all = 0
        max_visitors = 0
        min_visitors = 0
        
    if c == 387:
        v_ave_all = 0
        max_visitors = 0
        min_visitors = 0
        
    if c == 579:
        v_ave_all = 0
        max_visitors = 0
        min_visitors = 0
    
    # 以下の店舗は4月にしかデータがないので手動で平均値を算出した
    if c == 453:
        v_ave_all = 86.8
        max_visitors = 120
        min_visitors = 40
        v_ave_0 =  0
        v_ave_1 =  82.333
        v_ave_2 =  107
        v_ave_3 =  97.3
        v_ave_4 =  101.6
        v_ave_5 =  77
        v_ave_6 =  55.66
    
    v_ave_store = [[c, v_ave_all,v_ave_0, v_ave_1, v_ave_2, v_ave_3, v_ave_4, v_ave_5, v_ave_6, v_ave_h, max_visitors, min_visitors]]
    v_ave_temp = pd.DataFrame(v_ave_store, columns=columns)
    

        
    
    # 算出期間が短いと欠損が出る可能性があるので、修正する
    v_ave_temp.loc[v_ave_temp["v_ave_0"].isnull() == True, "v_ave_0"] = v_ave_all
    v_ave_temp.loc[v_ave_temp["v_ave_1"].isnull() == True, "v_ave_1"] = v_ave_all
    v_ave_temp.loc[v_ave_temp["v_ave_2"].isnull() == True, "v_ave_2"] = v_ave_all
    v_ave_temp.loc[v_ave_temp["v_ave_3"].isnull() == True, "v_ave_3"] = v_ave_all
    v_ave_temp.loc[v_ave_temp["v_ave_4"].isnull() == True, "v_ave_4"] = v_ave_all
    v_ave_temp.loc[v_ave_temp["v_ave_5"].isnull() == True, "v_ave_5"] = v_ave_all
    v_ave_temp.loc[v_ave_temp["v_ave_6"].isnull() == True, "v_ave_6"] = v_ave_all
    v_ave_temp.loc[v_ave_temp["v_ave_holiday"].isnull() == True, "v_ave_holiday"] = v_ave_all
    v_ave_temp
    
    v_ave = pd.concat([v_ave,v_ave_temp])

#インデックス振りなおす
v_ave = v_ave.reset_index(drop=True)
# v_ave.loc[v_ave.air_store_id==110]
v_ave

In [ ]:
# 欠損地のあるindexを探してくる
v_ave[pd.isnull(v_ave.v_ave_holiday)]
# v_ave.head(30)

# データ型をint64に変更

In [ ]:
v_ave["air_store_id"] = v_ave.air_store_id.astype("int64")
v_ave["max_visitors"] = v_ave.max_visitors.astype("int64")
v_ave["min_visitors"] = v_ave.min_visitors.astype("int64")

v_ave

# 来店者数が多い曜日のindexを特徴量に入れる
全ての行について、店舗で最も来店者数が多い曜日をindxで返す関数を使って、
特徴量をtest_xに追加する

# 店舗ごとに、来店者数が多い曜日のindexをもつDataFrameを作成する
全ての行について、店舗で最も来店者数が多い曜日をindxで返す関数を使って、
特徴量をtest_xに追加する

In [ ]:
def cal_max_ave_index(t):
    air_store_id, id2 = t
    #train_allから、来店者の数を算出する(testを含むとvisitors=0が入るのでtrain_allだけでやる)
    id_0 = train_all.loc[train_all.air_store_id == air_store_id]
    id_0_data = pd.DataFrame()
    id_0_data["visitors"] = id_0.visitors
    id_0_data["day_of_the_week"] = id_0.day_of_the_week
    id_0_data = id_0_data.reset_index(drop=True)
    id_0_data_ana = id_0_data.groupby(['day_of_the_week']).visitors.agg([sum,len])
    id_0_data_ana["ave"] = id_0_data_ana["sum"] // id_0_data_ana["len"]
    max_ave_index = id_0_data_ana['ave'].idxmax()
    return max_ave_index

# 先に、air_store_id一覧を用意しておく

In [ ]:
all_id = merge_data.air_store_id.unique()
all_id = pd.DataFrame(all_id, columns=["air_store_id"])
all_id = all_id.sort_values(by='air_store_id', ascending=True)
all_id = all_id.reset_index(drop=True)
all_id["id2"] = all_id["air_store_id"] * 2
all_id

# 平均来店者数の多い曜日(ピーク曜日)を調べる
air_store_id一覧に、最も平均来店者数が多い曜日を追加する

In [ ]:
%%time
all_id.loc[:,"max_ave_day"] = all_id[["air_store_id","id2"]].apply(cal_max_ave_index, axis=1)
all_id

# 店舗情報を利用する

In [ ]:
air_store_info = pd.read_csv("../input/recruit-restaurant-visitor-forecasting-data/air_store_info.csv")

store_le = pd.DataFrame()
store_le["air_store_id"] = air_store_info.air_store_id
genre_le = pd.DataFrame()
genre_le["air_genre_name"] = air_store_info.air_genre_name
area_le = pd.DataFrame()
area_le["air_area_name"] = air_store_info.air_area_name

area_le

# ラベルエンコーディングで店舗のジャンルを分ける
・store_id<br>
・ジャンル<br>
・エリア<br>
に対してそれぞれラベルエンコーディングする

In [ ]:
# store_idのラベルエンコーディング
for c in store_le:
    store_le[c] = le.transform(store_le[c])
store_le

# ジャンルのラベルエンコーディング
for c in genre_le:
    le_g = LabelEncoder()
    le_g.fit(genre_le[c])
    genre_le[c] = le_g.transform(genre_le[c])
genre_le

for c in area_le:
    le_a = LabelEncoder()
    le_a.fit(area_le[c])
    area_le[c] = le_a.transform(area_le[c])
area_le

# ラベルエンコーディングした特徴量を結合
これで、店舗IDごとに座標、エリア、ジャンルが入ったDataframe「air_store_info」を得られる

In [ ]:
air_store_info = air_store_info.drop(columns = "air_store_id")
air_store_info = air_store_info.join(store_le)
air_store_info = air_store_info.drop(columns = "air_genre_name")
air_store_info = air_store_info.join(genre_le)
air_store_info = air_store_info.drop(columns = "air_area_name")
air_store_info = air_store_info.join(area_le)
# 最も来店者の多い曜日情報と結合するためにソートしindexをつけなおす
air_store_info = air_store_info.sort_values(by='air_store_id')
# indexをつけなおす
air_store_info = air_store_info.reset_index(drop=True)
air_store_info

In [ ]:
all_id

# 平均来客数の多い曜日(ピーク曜日)と結合する
先に作成した、店舗ごとに最も平均来客数が多い曜日のDataFrame「all_id」と結合する

In [ ]:
air_store_info = air_store_info.rename(columns={'air_store_id': 'air_store_id_info'})
air_store_info = air_store_info.join(all_id)
air_store_info

# merge_dataに店舗情報を追加
vlookupみたいな操作をする。参照して追加するみたいな<br>
merge_dataには、trainもtestも、xもyも全部含まれる<br>
air_store_infoの店舗IDごとの情報を、各データに追加する

In [ ]:
merge_data = merge_data.merge(air_store_info, on="air_store_id",how="left")
merge_data

# 全てのIDを持つDataFrameを作成する

In [ ]:
all_id_only = pd.DataFrame()
all_id_only["id"] = all_id.air_store_id
all_id_only

# 前週の同じ曜日の来店者数を算出してlast_weekに持たせる
前週の同じ曜日の来店者数を、店舗ごとに求める。<br>
処理に30秒くらいかかる


In [ ]:
%%time
last_week = pd.DataFrame()

# for c in range(0,829,1):
for c in range(0, 829,1):
    
    last_week_0 = pd.DataFrame()
    last_week_1 = pd.DataFrame()
    last_week_2 = pd.DataFrame()
    last_week_3 = pd.DataFrame()
    last_week_4 = pd.DataFrame()
    last_week_5 = pd.DataFrame()
    last_week_6 = pd.DataFrame()
    
    store = merge_data.loc[merge_data.air_store_id == c]

    store_d = store.loc[store.day_of_the_week == 0]
    last_week_0["day_of_the_week"] = store_d.day_of_the_week
    last_week_0["today"] = store_d.visitors
    last_week_0["last_week"] = store_d["visitors"].shift(1)
    # 欠損はその店舗の平均来店者数で埋める
    ave = store_visitors['ave'][c]
    last_week_0 = last_week_0.fillna(ave)
    last_week = pd.concat([last_week, last_week_0])

    store_d = store.loc[store.day_of_the_week == 1]
    last_week_1["day_of_the_week"] = store_d.day_of_the_week
    last_week_1["today"] = store_d.visitors
    last_week_1["last_week"] = store_d["visitors"].shift(1)
        # 欠損はその店舗の平均来店者数で埋める
    ave = store_visitors['ave'][c]
    last_week_0 = last_week_0.fillna(ave)
    last_week = pd.concat([last_week, last_week_1])

    store_d = store.loc[store.day_of_the_week == 2]
    last_week_2["day_of_the_week"] = store_d.day_of_the_week
    last_week_2["today"] = store_d.visitors
    last_week_2["last_week"] = store_d["visitors"].shift(1)
        # 欠損はその店舗の平均来店者数で埋める
    ave = store_visitors['ave'][c]
    last_week_0 = last_week_0.fillna(ave)
    last_week = pd.concat([last_week, last_week_2])

    store_d = store.loc[store.day_of_the_week == 3]
    last_week_3["day_of_the_week"] = store_d.day_of_the_week
    last_week_3["today"] = store_d.visitors
    last_week_3["last_week"] = store_d["visitors"].shift(1)
        # 欠損はその店舗の平均来店者数で埋める
    ave = store_visitors['ave'][c]
    last_week_0 = last_week_0.fillna(ave)
    last_week = pd.concat([last_week, last_week_3])

    store_d = store.loc[store.day_of_the_week == 4]
    last_week_4["day_of_the_week"] = store_d.day_of_the_week
    last_week_4["today"] = store_d.visitors
    last_week_4["last_week"] = store_d["visitors"].shift(1)
        # 欠損はその店舗の平均来店者数で埋める
    ave = store_visitors['ave'][c]
    last_week_0 = last_week_0.fillna(ave)
    last_week = pd.concat([last_week, last_week_4])

    store_d = store.loc[store.day_of_the_week == 5]
    last_week_5["day_of_the_week"] = store_d.day_of_the_week
    last_week_5["today"] = store_d.visitors
    last_week_5["last_week"] = store_d["visitors"].shift(1)
        # 欠損はその店舗の平均来店者数で埋める
    ave = store_visitors['ave'][c]
    last_week_0 = last_week_0.fillna(ave)
    last_week = pd.concat([last_week, last_week_5])

    store_d = store.loc[store.day_of_the_week == 6]
    last_week_6["day_of_the_week"] = store_d.day_of_the_week
    last_week_6["today"] = store_d.visitors
    last_week_6["last_week"] = store_d["visitors"].shift(1)
        # 欠損はその店舗の平均来店者数で埋める
    ave = store_visitors['ave'][c]
    last_week_0 = last_week_0.fillna(ave)
    last_week = pd.concat([last_week, last_week_6])
    


last_week = last_week.sort_index()
last_week

In [ ]:
last_week.head(10)

# テストデータのvisitorsを埋める
事前に予測しておく。visitors_pre_predを入れる

In [ ]:
merge_data

In [ ]:
test_temp = merge_data.loc[merge_data.set == 'test']
test_temp

In [ ]:
v_ave

# v_aveで埋めていく

In [ ]:
# for c in range(0,829,1):
# for c in range(0,10,1):
# waringがうるさいので、withの範囲内にあるものはワーニング出さないようにする
import warnings
warnings.resetwarnings()
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    test_visitors = pd.DataFrame()
    for c in range(0,829,1):
        test_temp_store = test_temp.loc[test_temp.air_store_id == c]
        test_temp_store_0 = test_temp_store.loc[test_temp_store.day_of_the_week == 0]
        test_temp_store_0["visitors"] = v_ave.at[c,"v_ave_0"]
        test_visitors = pd.concat([test_visitors, test_temp_store_0])

        test_temp_store_1 = test_temp_store.loc[test_temp_store.day_of_the_week == 1]
        test_temp_store_1["visitors"] = v_ave.at[c,"v_ave_1"]
        test_visitors = pd.concat([test_visitors, test_temp_store_1])
        
        test_temp_store_2 = test_temp_store.loc[test_temp_store.day_of_the_week == 2]
        test_temp_store_2["visitors"] = v_ave.at[c,"v_ave_2"]
        test_visitors = pd.concat([test_visitors, test_temp_store_2])
        
        test_temp_store_3 = test_temp_store.loc[test_temp_store.day_of_the_week == 3]
        test_temp_store_3["visitors"] = v_ave.at[c,"v_ave_3"]
        test_visitors = pd.concat([test_visitors, test_temp_store_3])

        test_temp_store_4 = test_temp_store.loc[test_temp_store.day_of_the_week == 4]
        test_temp_store_4["visitors"] = v_ave.at[c,"v_ave_4"]
        test_visitors = pd.concat([test_visitors, test_temp_store_4])
        
        test_temp_store_5 = test_temp_store.loc[test_temp_store.day_of_the_week == 5]
        test_temp_store_5["visitors"] = v_ave.at[c,"v_ave_5"]
        test_visitors = pd.concat([test_visitors, test_temp_store_5])
        
        test_temp_store_6 = test_temp_store.loc[test_temp_store.day_of_the_week == 6]
        test_temp_store_6["visitors"] = v_ave.at[c,"v_ave_6"]
        test_visitors = pd.concat([test_visitors, test_temp_store_6])

test_visitors = test_visitors.sort_index()
test_visitors

In [ ]:
test_visitors[["air_store_id","day","visitors"]].tail(20)

# merge_dataに統合
testデータのvisitorsを、trainデータの店舗ごとの曜日ごと平均で与えてみる

In [ ]:
# 一応、直前のmerege_dataを補完
merge_data_temp = merge_data

# train_tempとtest_visitorsを結合
# 何も処理していないtrainデータと、visitorsを店舗の曜日ごとの平均で書き換えたtest_visitorsで書き換える
train_temp = merge_data.loc[merge_data.set == 'train']
merge_data = pd.concat([train_temp, test_visitors])
merge_data

In [ ]:
merge_data.loc[merge_data.air_store_id == c]

# ラグ特徴量の追加
 7,14,21日周期のラグ特徴量を算出してlag_allに持たせる

In [ ]:
%%time
store_all_7 = pd.DataFrame()
store_all_3 = pd.DataFrame()
store_all_14 = pd.DataFrame()
store_all_21 = pd.DataFrame()

for c in range(0,829,1):
    # cで指定した店舗idを持つDataFrameを拾ってくる(店舗別にラグ特徴量を得るために)
    store = merge_data.loc[merge_data.air_store_id == c]
    # 7周期前のlagを取得
    x_lag = pd.DataFrame()
    x_lag["lag7"] = store["visitors"].shift(0).rolling(window=7).mean()
    store = store.join(x_lag)
    # 欠損値は、その店舗の平均来店者数で埋める
    ave = store_visitors['ave'][c]
    store = store.fillna(ave)
    store_all_7 = pd.concat([store_all_7, store])
    
    
    store = merge_data.loc[merge_data.air_store_id == c]
    # 直近3日の平均
    x_lag = pd.DataFrame()
    x_lag["lag3"] = store["visitors"].shift(0).rolling(window=3).mean()
    store = store.join(x_lag)
    # 欠損値は、その店舗の平均来店者数で埋める
    ave = store_visitors['ave'][c]
    store = store.fillna(ave)
    store_all_3 = pd.concat([store_all_3, store])

    
    # cで指定した店舗idを持つDataFrameを拾ってくる(店舗別にラグ特徴量を得るために)
    store = merge_data.loc[merge_data.air_store_id == c]
    # 14周期前のlagを取得
    x_lag = pd.DataFrame()
    x_lag["lag14"] = store["visitors"].shift(7).rolling(window=7).mean()
    store = store.join(x_lag)
    # 欠損値は、その店舗の平均来店者数で埋める
    ave = store_visitors['ave'][c]
    store = store.fillna(ave)
    store_all_14 = pd.concat([store_all_14, store])
    
    # cで指定した店舗idを持つDataFrameを拾ってくる(店舗別にラグ特徴量を得るために)
    store = merge_data.loc[merge_data.air_store_id == c]
    # 21周期前のlagを取得
    x_lag = pd.DataFrame()
    x_lag["lag21"] = store["visitors"].shift(14).rolling(window=7).mean()
    store = store.join(x_lag)
    # 欠損値は、その店舗の平均来店者数で埋める
    ave = store_visitors['ave'][c]
    store = store.fillna(ave)
    store_all_21 = pd.concat([store_all_21, store])
    


In [ ]:
# idでソートしなおす
store_all_7 = store_all_7.sort_index()
store_all_3 = store_all_3.sort_index()
store_all_14 = store_all_14.sort_index()
store_all_21 = store_all_21.sort_index()
# ラグ特徴量を結合してlag_all
lag_all = pd.DataFrame()
lag_all["lag7"] = store_all_7.lag7
lag_all["lag3"] = store_all_3.lag3
lag_all["lag14"] = store_all_14.lag14
lag_all["lag21"] = store_all_21.lag21
lag_all.head(30)

# ラグ特徴量をmerge_dataに追加

In [ ]:
merge_data = merge_data.join(lag_all)
merge_data = merge_data.join(last_week, rsuffix='_last')
merge_data

In [ ]:
ckm = merge_data.loc[merge_data.set == "test"]
ckm.head(30)

In [ ]:
ckm[['air_store_id', 'visitors', "month", "day","set","lag7","lag14","lag21"]].tail(30)

# 各店舗の来店者数情報も追加
v_aveで算出した、各店舗の情報を追加

In [ ]:
merge_data = merge_data.merge(v_ave, on="air_store_id",how="left")
merge_data

In [ ]:
# リークする危険があるのでか必ずtodayをdropして消す
merge_data = merge_data.drop(columns='today')
merge_data = merge_data.drop(columns='day_of_the_week_last')
merge_data = merge_data.drop(columns='id2')

# 全期間のデータをmerge_data_allとして残す

In [ ]:
#期間短くする前のmerge_dataは、merge_data_all(全期間のデータ)として残す
merge_data_all = merge_data
merge_data_all

In [ ]:
# merge_data_allへはここで修復できる
merge_data = merge_data_all
merge_data

# 学習を早くするために、データを絞る
データ量が多く、xgboostの学習に30minかかるので、2017年4月-5月だけにデータを絞る。<br>
28万行→4.7万行に減らせる。トレーニングデータの数で言えば、25.5万→2.2万<br><br>
【学習データを絞らない場合は、ここをコメントアウト】

In [ ]:
# #2017年の3-5月のデータだけを使う
# merge_data = merge_data.loc[(merge_data.year == 2017)]
# merge_data = merge_data.loc[(merge_data.month == 1)
#                             | (merge_data.month == 2)
#                             | (merge_data.month == 3)
#                             | (merge_data.month == 4)
#                             | (merge_data.month == 5)]
# merge_data

# 4/1-4/22までをvalidationデータとする
4/23以降はテストデータ

In [ ]:
# merge_data.loc[(merge_data['month'] == 4) & (merge_data['day'] <= 22), 'set'] = 'va'
# merge_data

In [ ]:
merge_data.loc[merge_data.set == "va"]

# 特徴量を落としてみる
意味のある特徴量、ない特徴量を整理したい(基本的に全部あった方がいい認識だが)
* ラグ特徴量lag7-21をとらないだけでRMSLEが1.10→1.57に下がった


In [ ]:
# merge_data = merge_data.drop(columns="lag7")
# merge_data = merge_data.drop(columns="lag14")
# merge_data = merge_data.drop(columns="lag21")

# 以上で、特徴量エンジニアリングは終了

# データをtrain_fitとtest_fitに分割する
model学習に使うデータとして分割する。
来店者数が0より以上の場合、trainデータとして認識する

In [ ]:
train_fit = pd.DataFrame()
train_fit = merge_data.loc[merge_data.set == "train"]
train_fit_y = pd.DataFrame()
train_fit_y["visitors"] = train_fit.visitors
train_fit_x = train_fit.drop(columns="visitors")
train_fit_x = train_fit_x.drop(columns="set")

test_fit = pd.DataFrame()
test_fit = merge_data.loc[merge_data.set == "test"]
test_fit_x = test_fit.drop(columns="visitors")
test_fit_x = test_fit_x.drop(columns="set")

In [ ]:
va_fit = pd.DataFrame()
va_fit = merge_data.loc[merge_data.set == "va"]
va_fit_y = pd.DataFrame()
va_fit_y["visitors"] = va_fit.visitors
va_fit_x = va_fit.drop(columns="visitors")
va_fit_x = va_fit_x.drop(columns="set")

In [ ]:
train_fit_x

In [ ]:
train_fit_y

In [ ]:
test_fit_x

In [ ]:
test_fit_x[["air_store_id","day_of_the_week","day","lag7","lag14","lag21"]]

In [ ]:
va_fit_x

In [ ]:
va_fit_y

# xgbのパラメータを決定

In [ ]:
from xgboost import XGBClassifier as XGB
from sklearn.metrics import mean_squared_log_error

Model = XGB(n_estimators=100, random_state=71,max_depth=4,colsample_bylevel=0.4,
           gamma=0.0,alpha=0.0, min_child_weight=4,subsample=0.8,colsample_bytree=0.95
           )

# モデル学習して予測、RMSLEを算出
* xgboostで学習させる
* 4月(validationデータ)のvisitorsをhold-outで予測
* RMSLEを計算する

In [ ]:
%%time
Model.fit(train_fit_x, train_fit_y)

# hold-outでのvalidationデータの予測
時系列的にうしろのデータを使って単純なバリデーション<br>
バリデーションデータない場合はコメントアウト

In [ ]:
# # モデルで予測する
# pred = Model.predict(va_fit_x)

# # 予測値からRMSLEを算出する
# score = np.sqrt(mean_squared_log_error(va_fit_y, pred))
# print("---------------RMSLE-score----------------")
# score

# テストデータの予測

In [ ]:
pred_test = Model.predict(test_fit_x)
pred_test

# 店舗110と663は、全て0で埋める
と思ったが、testデータ中に110,663は存在しないらいい

In [ ]:
# submission_fill0 = pd.DataFrame()
# submission_fill0["visitors"] = pred
# test_fit_x_reindex = test_fit_x.reset_index()
# submission_fill0 = submission_fill0.join(test_fit_x_reindex["air_store_id"])

# submission_fill0.loc[submission_fill0['air_store_id'] == 663, 'visitors'] = 0
# submission_fill0.loc[submission_fill0.air_store_id == 663]

# 提出データの作成

In [ ]:
submission = pd.DataFrame()
submission["id"] = test.id
submission["visitors"] = pred_test

In [ ]:
submission.to_csv("submission_21.csv", index=False)

# 来店者平均値から算出
test_fit_xのラグ特徴量を計算するために入れたvisitorsの値をそのまま出力する。<br>
3-4月の曜日ごとの来店者数の平均をとってtestデータに当てはめると、<br>
Priveateで0.55572のスコアが得られる

In [ ]:
submission_ave = pd.DataFrame()
test_visitors_reindex = pd.DataFrame()
submission_ave["id"] = test.id
test_visitors_reindex = test_visitors.reset_index(drop=True)
submission_ave["visitors"] = test_visitors_reindex.visitors
submission_ave

In [ ]:
submission_ave.to_csv("submission_test_v.csv", index=False)

In [ ]:
pred_v

In [ ]:
va_fit_y

# 予測データと実際のデータを比較するデータフレームを作る

In [ ]:
compare_v = va_fit_x
compare_v = compare_v.join(va_fit_y)
compare_v_reindex = compare_v.reset_index(drop=True)
pred_v["visitors_pred"] = pd.DataFrame(pred)
compare_v_reindex = compare_v_reindex.join(pred_v)
compare_v_reindex["delta"] = compare_v_reindex["visitors_pred"] - compare_v_reindex["visitors"]
compare_v_reindex["delta_abs"] = abs(compare_v_reindex["delta"])
ck = compare_v_reindex[["air_store_id","month","day","day_of_the_week","visitors_pred","visitors","delta","delta_abs"]]

# 誤差の大きいデータを表示する

In [ ]:
ck_bad = ck.sort_values(by='delta_abs', ascending=False).head(100)
ck_bad.sort_values(by='air_store_id', ascending=False).tail(50)

In [ ]:
sns.distplot(
    ck_bad['visitors'], bins=100, color='#123456', label='data',
    kde=False,
    rug=False
)

# ID=453の店舗の来店者数の経緯を見る

In [ ]:
visitor_453 = merge_data.loc[merge_data.air_store_id == 453]
visitor_453 = visitor_453[["air_store_id","month","day","day_of_the_week","visitors"]]
visitor_453 = visitor_453.reset_index()
visitor_453 = visitor_453.drop(columns = "index")
visitor_453 = visitor_453.reset_index()
visitor_453

In [ ]:
sns.lineplot(x="index", y="visitors", data=visitor_453)

# 各曜日で来店者数の情報を表示

In [ ]:
visitor_453.loc[visitor_453.day_of_the_week == 6]

In [ ]:
visitor_371 = merge_data.loc[merge_data.air_store_id == 371]
visitor_371 = visitor_371[["air_store_id","month","day","day_of_the_week","visitors"]]
visitor_371 = visitor_371.reset_index()
visitor_371 = visitor_371.drop(columns = "index")
visitor_371 = visitor_371.reset_index()
visitor_371

In [ ]:
sns.lineplot(x="index", y="visitors", data=visitor_371)

In [ ]:
visitor_371.loc[visitor_371.day_of_the_week == 4]

# 可視化する

In [ ]:
sns.scatterplot(x=va_fit_y['visitors'], y=pred)

In [ ]:
va_fit_x

In [ ]:
pred

In [ ]:
column = ["visitors_pred"]
pred_d = pd.DataFrame(pred, columns=column)
pred_d

In [ ]:
pred_d[pd.isnull(pred_d.visitors_pred)]

In [ ]:
va_fit_y

In [ ]:
va_fit_x

In [ ]:
va_fit_y_reset = va_fit_y.reset_index()
va_df = va_fit_x.reset_index()
va_df = va_df.join(va_fit_y_reset,lsuffix='_y')
va_df = va_df.join(pred_d)
va_df

In [ ]:
va_df["delta"] = va_df["visitors"] - va_df["visitors_pred"]

In [ ]:
va_df = va_df.drop(columns = "index_y")
va_df = va_df.drop(columns = "air_store_id_info")
va_df.head(30)

In [ ]:
plot = sns.scatterplot(x=va_df['visitors'], y=va_df["visitors_pred"])
plot = sns.scatterplot(x=va_df['visitors'], y=va_df["visitors"])

In [ ]:
sns.scatterplot(x=va_df['visitors'], y=va_df["delta"])

# 4月の来店者数を予測する
まず、4月の来店者数を予測する。

In [ ]:
%%time

for n in range(23, 31, 1):
    
    # 特定の日付について、予測値をtest_predに追加する
    test_pred = test_fit_x.loc[(test_fit_x.month == 4) & (test_fit_x.day == n)]
    pred = Model.predict(test_pred)
    test_pred["visitors"] = pred
    test_pred_0 = pd.DataFrame()
    test_pred_0["visitors_predict"] = test_pred.visitors

    # visitors_predictをmerge_dataに結合する
    merge_data = merge_data.merge(test_pred_0, left_index = True, right_index = True, how="left")

    # 追加した、visitors_predictのうち、NaNじゃない列は、visitorsの値にvisitors_predictの値を代入する
    merge_data['visitors'] = merge_data['visitors'].where(merge_data['visitors_predict'].isnull() == True, merge_data['visitors_predict'])

    # その後、visitors_predictは削除する
    merge_data = merge_data.drop(columns ="visitors_predict")

    # 2017年4月以降のデータで、テストデータのラグ特徴量を再度計算する
    merge_data_trim = merge_data.loc[(merge_data.month >= 4) & (merge_data.year == 2017)]

    # 既にある特徴量は一旦削除する
    merge_data_trim = merge_data_trim.drop(['lag7', 'lag14', 'lag21', 'last_week'], axis=1)

    # 前週の同じ曜日の来店者数を取得する
    last_week = pd.DataFrame()

    # for c in range(0,829,1):
    for c in range(0, 829,1):

        last_week_0 = pd.DataFrame()
        last_week_1 = pd.DataFrame()
        last_week_2 = pd.DataFrame()
        last_week_3 = pd.DataFrame()
        last_week_4 = pd.DataFrame()
        last_week_5 = pd.DataFrame()
        last_week_6 = pd.DataFrame()

        store = merge_data_trim.loc[merge_data_trim.air_store_id == c]

        store_d = store.loc[store.day_of_the_week == 0]
        last_week_0["day_of_the_week"] = store_d.day_of_the_week
        last_week_0["today"] = store_d.visitors
        last_week_0["last_week"] = store_d["visitors"].shift(1)
        # 欠損はその店舗の平均来店者数で埋める
        ave = store_visitors['ave'][c]
        last_week_0 = last_week_0.fillna(ave)
        last_week = pd.concat([last_week, last_week_0])

        store_d = store.loc[store.day_of_the_week == 1]
        last_week_1["day_of_the_week"] = store_d.day_of_the_week
        last_week_1["today"] = store_d.visitors
        last_week_1["last_week"] = store_d["visitors"].shift(1)
            # 欠損はその店舗の平均来店者数で埋める
        ave = store_visitors['ave'][c]
        last_week_0 = last_week_0.fillna(ave)
        last_week = pd.concat([last_week, last_week_1])

        store_d = store.loc[store.day_of_the_week == 2]
        last_week_2["day_of_the_week"] = store_d.day_of_the_week
        last_week_2["today"] = store_d.visitors
        last_week_2["last_week"] = store_d["visitors"].shift(1)
            # 欠損はその店舗の平均来店者数で埋める
        ave = store_visitors['ave'][c]
        last_week_0 = last_week_0.fillna(ave)
        last_week = pd.concat([last_week, last_week_2])

        store_d = store.loc[store.day_of_the_week == 3]
        last_week_3["day_of_the_week"] = store_d.day_of_the_week
        last_week_3["today"] = store_d.visitors
        last_week_3["last_week"] = store_d["visitors"].shift(1)
            # 欠損はその店舗の平均来店者数で埋める
        ave = store_visitors['ave'][c]
        last_week_0 = last_week_0.fillna(ave)
        last_week = pd.concat([last_week, last_week_3])

        store_d = store.loc[store.day_of_the_week == 4]
        last_week_4["day_of_the_week"] = store_d.day_of_the_week
        last_week_4["today"] = store_d.visitors
        last_week_4["last_week"] = store_d["visitors"].shift(1)
            # 欠損はその店舗の平均来店者数で埋める
        ave = store_visitors['ave'][c]
        last_week_0 = last_week_0.fillna(ave)
        last_week = pd.concat([last_week, last_week_4])

        store_d = store.loc[store.day_of_the_week == 5]
        last_week_5["day_of_the_week"] = store_d.day_of_the_week
        last_week_5["today"] = store_d.visitors
        last_week_5["last_week"] = store_d["visitors"].shift(1)
            # 欠損はその店舗の平均来店者数で埋める
        ave = store_visitors['ave'][c]
        last_week_0 = last_week_0.fillna(ave)
        last_week = pd.concat([last_week, last_week_5])

        store_d = store.loc[store.day_of_the_week == 6]
        last_week_6["day_of_the_week"] = store_d.day_of_the_week
        last_week_6["today"] = store_d.visitors
        last_week_6["last_week"] = store_d["visitors"].shift(1)
            # 欠損はその店舗の平均来店者数で埋める
        ave = store_visitors['ave'][c]
        last_week_0 = last_week_0.fillna(ave)
        last_week = pd.concat([last_week, last_week_6])

    last_week = last_week.sort_index()

    # 7,14,21日のラグ特徴量を取得する
    store_all_7 = pd.DataFrame()
    store_all_14 = pd.DataFrame()
    store_all_21 = pd.DataFrame()

    for c in range(0,829,1):
        # cで指定した店舗idを持つDataFrameを拾ってくる(店舗別にラグ特徴量を得るために)
        store = merge_data_trim.loc[merge_data_trim.air_store_id == c]
        # 7周期前のlagを取得
        x_lag = pd.DataFrame()
        x_lag["lag7"] = store["visitors"].shift(0).rolling(window=7).mean()
        store = store.join(x_lag)
        # 欠損値は、その店舗の平均来店者数で埋める
        ave = store_visitors['ave'][c]
        store = store.fillna(ave)
        store_all_7 = pd.concat([store_all_7, store])


        # cで指定した店舗idを持つDataFrameを拾ってくる(店舗別にラグ特徴量を得るために)
        store = merge_data_trim.loc[merge_data_trim.air_store_id == c]
        # 14周期前のlagを取得
        x_lag = pd.DataFrame()
        x_lag["lag14"] = store["visitors"].shift(7).rolling(window=7).mean()
        store = store.join(x_lag)
        # 欠損値は、その店舗の平均来店者数で埋める
        ave = store_visitors['ave'][c]
        store = store.fillna(ave)
        store_all_14 = pd.concat([store_all_14, store])

        # cで指定した店舗idを持つDataFrameを拾ってくる(店舗別にラグ特徴量を得るために)
        store = merge_data_trim.loc[merge_data_trim.air_store_id == c]
        # 21周期前のlagを取得
        x_lag = pd.DataFrame()
        x_lag["lag21"] = store["visitors"].shift(14).rolling(window=7).mean()
        store = store.join(x_lag)
        # 欠損値は、その店舗の平均来店者数で埋める
        ave = store_visitors['ave'][c]
        store = store.fillna(ave)
        store_all_21 = pd.concat([store_all_21, store])

    # idでソートしなおす
    store_all_7 = store_all_7.sort_index()
    store_all_14 = store_all_14.sort_index()
    store_all_21 = store_all_21.sort_index()
    # ラグ特徴量を結合してlag_all
    lag_all = pd.DataFrame()
    lag_all["lag7"] = store_all_7.lag7
    lag_all["lag14"] = store_all_14.lag14
    lag_all["lag21"] = store_all_21.lag21

    # 特徴量を追加する
    merge_data_trim = merge_data_trim.join(lag_all)
    merge_data_trim = merge_data_trim.join(last_week, rsuffix='_last')
    
    # 予測した値を残して、ループ処理後に参照できるようにする
    pred_visitors_all = merge_data_trim.loc[:,["air_store_id","month","day", 'visitors','last_week']]

    # リークする危険があるのでか必ずtodayをdropして消す
    merge_data_trim = merge_data_trim.drop(columns='today')
    merge_data_trim = merge_data_trim.drop(columns='visitors')
    merge_data_trim = merge_data_trim.drop(columns='day_of_the_week_last')
    

    # test_fit_xを初期化する
    test_fit_x = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)

    #　テストデータのみ切り分けて、次の予測に用いる
    test_fit_x = merge_data_trim.loc[merge_data_trim.set == "test"]
    test_fit_x = test_fit_x.drop(columns='set')
    
print("done")

# 5月のデータを予測する(private_leaderboardと思われる)
基本的に、4月と同じモデルで予測する

In [ ]:
%%time

for n in range(1, 32, 1):
    
    # 特定の日付について、予測値をtest_predに追加する
    test_pred = test_fit_x.loc[(test_fit_x.month == 5) & (test_fit_x.day == n)]
    pred = Model.predict(test_pred)
    test_pred["visitors"] = pred
    test_pred_0 = pd.DataFrame()
    test_pred_0["visitors_predict"] = test_pred.visitors

    # visitors_predictをmerge_dataに結合する
    merge_data = merge_data.merge(test_pred_0, left_index = True, right_index = True, how="left")

    # 追加した、visitors_predictのうち、NaNじゃない列は、visitorsの値にvisitors_predictの値を代入する
    merge_data['visitors'] = merge_data['visitors'].where(merge_data['visitors_predict'].isnull() == True, merge_data['visitors_predict'])

    # その後、visitors_predictは削除する
    merge_data = merge_data.drop(columns ="visitors_predict")

    # 2017年4月以降のデータで、テストデータのラグ特徴量を再度計算する
    merge_data_trim = merge_data.loc[(merge_data.month >= 4) & (merge_data.year == 2017)]

    # 既にある特徴量は一旦削除する
    merge_data_trim = merge_data_trim.drop(['lag7', 'lag14', 'lag21', 'last_week'], axis=1)

    # 前週の同じ曜日の来店者数を取得する
    last_week = pd.DataFrame()

    # for c in range(0,829,1):
    for c in range(0, 829,1):

        last_week_0 = pd.DataFrame()
        last_week_1 = pd.DataFrame()
        last_week_2 = pd.DataFrame()
        last_week_3 = pd.DataFrame()
        last_week_4 = pd.DataFrame()
        last_week_5 = pd.DataFrame()
        last_week_6 = pd.DataFrame()

        store = merge_data_trim.loc[merge_data_trim.air_store_id == c]

        store_d = store.loc[store.day_of_the_week == 0]
        last_week_0["day_of_the_week"] = store_d.day_of_the_week
        last_week_0["today"] = store_d.visitors
        last_week_0["last_week"] = store_d["visitors"].shift(1)
        # 欠損はその店舗の平均来店者数で埋める
        ave = store_visitors['ave'][c]
        last_week_0 = last_week_0.fillna(ave)
        last_week = pd.concat([last_week, last_week_0])

        store_d = store.loc[store.day_of_the_week == 1]
        last_week_1["day_of_the_week"] = store_d.day_of_the_week
        last_week_1["today"] = store_d.visitors
        last_week_1["last_week"] = store_d["visitors"].shift(1)
            # 欠損はその店舗の平均来店者数で埋める
        ave = store_visitors['ave'][c]
        last_week_0 = last_week_0.fillna(ave)
        last_week = pd.concat([last_week, last_week_1])

        store_d = store.loc[store.day_of_the_week == 2]
        last_week_2["day_of_the_week"] = store_d.day_of_the_week
        last_week_2["today"] = store_d.visitors
        last_week_2["last_week"] = store_d["visitors"].shift(1)
            # 欠損はその店舗の平均来店者数で埋める
        ave = store_visitors['ave'][c]
        last_week_0 = last_week_0.fillna(ave)
        last_week = pd.concat([last_week, last_week_2])

        store_d = store.loc[store.day_of_the_week == 3]
        last_week_3["day_of_the_week"] = store_d.day_of_the_week
        last_week_3["today"] = store_d.visitors
        last_week_3["last_week"] = store_d["visitors"].shift(1)
            # 欠損はその店舗の平均来店者数で埋める
        ave = store_visitors['ave'][c]
        last_week_0 = last_week_0.fillna(ave)
        last_week = pd.concat([last_week, last_week_3])

        store_d = store.loc[store.day_of_the_week == 4]
        last_week_4["day_of_the_week"] = store_d.day_of_the_week
        last_week_4["today"] = store_d.visitors
        last_week_4["last_week"] = store_d["visitors"].shift(1)
            # 欠損はその店舗の平均来店者数で埋める
        ave = store_visitors['ave'][c]
        last_week_0 = last_week_0.fillna(ave)
        last_week = pd.concat([last_week, last_week_4])

        store_d = store.loc[store.day_of_the_week == 5]
        last_week_5["day_of_the_week"] = store_d.day_of_the_week
        last_week_5["today"] = store_d.visitors
        last_week_5["last_week"] = store_d["visitors"].shift(1)
            # 欠損はその店舗の平均来店者数で埋める
        ave = store_visitors['ave'][c]
        last_week_0 = last_week_0.fillna(ave)
        last_week = pd.concat([last_week, last_week_5])

        store_d = store.loc[store.day_of_the_week == 6]
        last_week_6["day_of_the_week"] = store_d.day_of_the_week
        last_week_6["today"] = store_d.visitors
        last_week_6["last_week"] = store_d["visitors"].shift(1)
            # 欠損はその店舗の平均来店者数で埋める
        ave = store_visitors['ave'][c]
        last_week_0 = last_week_0.fillna(ave)
        last_week = pd.concat([last_week, last_week_6])

    last_week = last_week.sort_index()

    # 7,14,21日のラグ特徴量を取得する
    store_all_7 = pd.DataFrame()
    store_all_14 = pd.DataFrame()
    store_all_21 = pd.DataFrame()

    for c in range(0,829,1):
        # cで指定した店舗idを持つDataFrameを拾ってくる(店舗別にラグ特徴量を得るために)
        store = merge_data_trim.loc[merge_data_trim.air_store_id == c]
        # 7周期前のlagを取得
        x_lag = pd.DataFrame()
        x_lag["lag7"] = store["visitors"].shift(0).rolling(window=7).mean()
        store = store.join(x_lag)
        # 欠損値は、その店舗の平均来店者数で埋める
        ave = store_visitors['ave'][c]
        store = store.fillna(ave)
        store_all_7 = pd.concat([store_all_7, store])


        # cで指定した店舗idを持つDataFrameを拾ってくる(店舗別にラグ特徴量を得るために)
        store = merge_data_trim.loc[merge_data_trim.air_store_id == c]
        # 14周期前のlagを取得
        x_lag = pd.DataFrame()
        x_lag["lag14"] = store["visitors"].shift(7).rolling(window=7).mean()
        store = store.join(x_lag)
        # 欠損値は、その店舗の平均来店者数で埋める
        ave = store_visitors['ave'][c]
        store = store.fillna(ave)
        store_all_14 = pd.concat([store_all_14, store])

        # cで指定した店舗idを持つDataFrameを拾ってくる(店舗別にラグ特徴量を得るために)
        store = merge_data_trim.loc[merge_data_trim.air_store_id == c]
        # 21周期前のlagを取得
        x_lag = pd.DataFrame()
        x_lag["lag21"] = store["visitors"].shift(14).rolling(window=7).mean()
        store = store.join(x_lag)
        # 欠損値は、その店舗の平均来店者数で埋める
        ave = store_visitors['ave'][c]
        store = store.fillna(ave)
        store_all_21 = pd.concat([store_all_21, store])

    # idでソートしなおす
    store_all_7 = store_all_7.sort_index()
    store_all_14 = store_all_14.sort_index()
    store_all_21 = store_all_21.sort_index()
    # ラグ特徴量を結合してlag_all
    lag_all = pd.DataFrame()
    lag_all["lag7"] = store_all_7.lag7
    lag_all["lag14"] = store_all_14.lag14
    lag_all["lag21"] = store_all_21.lag21

    # 特徴量を追加する
    merge_data_trim = merge_data_trim.join(lag_all)
    merge_data_trim = merge_data_trim.join(last_week, rsuffix='_last')
    
    # 予測した値を残して、ループ処理後に参照できるようにする
    pred_visitors_all = merge_data_trim.loc[:,["set", "air_store_id","month","day", 'visitors','last_week']]

    # リークする危険があるのでか必ずtodayをdropして消す
    merge_data_trim = merge_data_trim.drop(columns='today')
    merge_data_trim = merge_data_trim.drop(columns='visitors')
    merge_data_trim = merge_data_trim.drop(columns='day_of_the_week_last')
    

    # test_fit_xを初期化する
    test_fit_x = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)

    #　テストデータのみ切り分けて、次の予測に用いる
    test_fit_x = merge_data_trim.loc[merge_data_trim.set == "test"]
    test_fit_x = test_fit_x.drop(columns='set')
    
print("done")

In [ ]:
pred = pred_visitors_all.loc[(pred_visitors_all.set == "test")]
pred

# 提出データを用意する
submissionデータの判定にためには、columns名「visitors」が入っている必要がある

In [ ]:
%%time

for n in range(23, 31, 1):
    
    # 特定の日付について、予測値をtest_predに追加する
    test_pred = test_fit_x.loc[(test_fit_x.month == 4) & (test_fit_x.day == n)]
    pred = Model.predict(test_pred)
    test_pred["visitors"] = pred
    test_pred_0 = pd.DataFrame()
    test_pred_0["visitors_predict"] = test_pred.visitors

    # visitors_predictをmerge_dataに結合する
    merge_data = merge_data.merge(test_pred_0, left_index = True, right_index = True, how="left")

    # 追加した、visitors_predictのうち、NaNじゃない列は、visitorsの値にvisitors_predictの値を代入する
    merge_data['visitors'] = merge_data['visitors'].where(merge_data['visitors_predict'].isnull() == True, merge_data['visitors_predict'])

    # その後、visitors_predictは削除する
    merge_data = merge_data.drop(columns ="visitors_predict")

    # 2017年4月以降のデータで、テストデータのラグ特徴量を再度計算する
    merge_data_trim = merge_data.loc[(merge_data.month >= 4) & (merge_data.year == 2017)]

    # 既にある特徴量は一旦削除する
    merge_data_trim = merge_data_trim.drop(['lag7', 'lag14', 'lag21', 'last_week'], axis=1)

    # 前週の同じ曜日の来店者数を取得する
    last_week = pd.DataFrame()

    # for c in range(0,829,1):
    for c in range(0, 829,1):

        last_week_0 = pd.DataFrame()
        last_week_1 = pd.DataFrame()
        last_week_2 = pd.DataFrame()
        last_week_3 = pd.DataFrame()
        last_week_4 = pd.DataFrame()
        last_week_5 = pd.DataFrame()
        last_week_6 = pd.DataFrame()

        store = merge_data_trim.loc[merge_data_trim.air_store_id == c]

        store_d = store.loc[store.day_of_the_week == 0]
        last_week_0["day_of_the_week"] = store_d.day_of_the_week
        last_week_0["today"] = store_d.visitors
        last_week_0["last_week"] = store_d["visitors"].shift(1)
        # 欠損はその店舗の平均来店者数で埋める
        ave = store_visitors['ave'][c]
        last_week_0 = last_week_0.fillna(ave)
        last_week = pd.concat([last_week, last_week_0])

        store_d = store.loc[store.day_of_the_week == 1]
        last_week_1["day_of_the_week"] = store_d.day_of_the_week
        last_week_1["today"] = store_d.visitors
        last_week_1["last_week"] = store_d["visitors"].shift(1)
            # 欠損はその店舗の平均来店者数で埋める
        ave = store_visitors['ave'][c]
        last_week_0 = last_week_0.fillna(ave)
        last_week = pd.concat([last_week, last_week_1])

        store_d = store.loc[store.day_of_the_week == 2]
        last_week_2["day_of_the_week"] = store_d.day_of_the_week
        last_week_2["today"] = store_d.visitors
        last_week_2["last_week"] = store_d["visitors"].shift(1)
            # 欠損はその店舗の平均来店者数で埋める
        ave = store_visitors['ave'][c]
        last_week_0 = last_week_0.fillna(ave)
        last_week = pd.concat([last_week, last_week_2])

        store_d = store.loc[store.day_of_the_week == 3]
        last_week_3["day_of_the_week"] = store_d.day_of_the_week
        last_week_3["today"] = store_d.visitors
        last_week_3["last_week"] = store_d["visitors"].shift(1)
            # 欠損はその店舗の平均来店者数で埋める
        ave = store_visitors['ave'][c]
        last_week_0 = last_week_0.fillna(ave)
        last_week = pd.concat([last_week, last_week_3])

        store_d = store.loc[store.day_of_the_week == 4]
        last_week_4["day_of_the_week"] = store_d.day_of_the_week
        last_week_4["today"] = store_d.visitors
        last_week_4["last_week"] = store_d["visitors"].shift(1)
            # 欠損はその店舗の平均来店者数で埋める
        ave = store_visitors['ave'][c]
        last_week_0 = last_week_0.fillna(ave)
        last_week = pd.concat([last_week, last_week_4])

        store_d = store.loc[store.day_of_the_week == 5]
        last_week_5["day_of_the_week"] = store_d.day_of_the_week
        last_week_5["today"] = store_d.visitors
        last_week_5["last_week"] = store_d["visitors"].shift(1)
            # 欠損はその店舗の平均来店者数で埋める
        ave = store_visitors['ave'][c]
        last_week_0 = last_week_0.fillna(ave)
        last_week = pd.concat([last_week, last_week_5])

        store_d = store.loc[store.day_of_the_week == 6]
        last_week_6["day_of_the_week"] = store_d.day_of_the_week
        last_week_6["today"] = store_d.visitors
        last_week_6["last_week"] = store_d["visitors"].shift(1)
            # 欠損はその店舗の平均来店者数で埋める
        ave = store_visitors['ave'][c]
        last_week_0 = last_week_0.fillna(ave)
        last_week = pd.concat([last_week, last_week_6])

    last_week = last_week.sort_index()

    # 7,14,21日のラグ特徴量を取得する
    store_all_7 = pd.DataFrame()
    store_all_14 = pd.DataFrame()
    store_all_21 = pd.DataFrame()

    for c in range(0,829,1):
        # cで指定した店舗idを持つDataFrameを拾ってくる(店舗別にラグ特徴量を得るために)
        store = merge_data_trim.loc[merge_data_trim.air_store_id == c]
        # 7周期前のlagを取得
        x_lag = pd.DataFrame()
        x_lag["lag7"] = store["visitors"].shift(0).rolling(window=7).mean()
        store = store.join(x_lag)
        # 欠損値は、その店舗の平均来店者数で埋める
        ave = store_visitors['ave'][c]
        store = store.fillna(ave)
        store_all_7 = pd.concat([store_all_7, store])


        # cで指定した店舗idを持つDataFrameを拾ってくる(店舗別にラグ特徴量を得るために)
        store = merge_data_trim.loc[merge_data_trim.air_store_id == c]
        # 14周期前のlagを取得
        x_lag = pd.DataFrame()
        x_lag["lag14"] = store["visitors"].shift(7).rolling(window=7).mean()
        store = store.join(x_lag)
        # 欠損値は、その店舗の平均来店者数で埋める
        ave = store_visitors['ave'][c]
        store = store.fillna(ave)
        store_all_14 = pd.concat([store_all_14, store])

        # cで指定した店舗idを持つDataFrameを拾ってくる(店舗別にラグ特徴量を得るために)
        store = merge_data_trim.loc[merge_data_trim.air_store_id == c]
        # 21周期前のlagを取得
        x_lag = pd.DataFrame()
        x_lag["lag21"] = store["visitors"].shift(14).rolling(window=7).mean()
        store = store.join(x_lag)
        # 欠損値は、その店舗の平均来店者数で埋める
        ave = store_visitors['ave'][c]
        store = store.fillna(ave)
        store_all_21 = pd.concat([store_all_21, store])

    # idでソートしなおす
    store_all_7 = store_all_7.sort_index()
    store_all_14 = store_all_14.sort_index()
    store_all_21 = store_all_21.sort_index()
    # ラグ特徴量を結合してlag_all
    lag_all = pd.DataFrame()
    lag_all["lag7"] = store_all_7.lag7
    lag_all["lag14"] = store_all_14.lag14
    lag_all["lag21"] = store_all_21.lag21

    # 特徴量を追加する
    merge_data_trim = merge_data_trim.join(lag_all)
    merge_data_trim = merge_data_trim.join(last_week, rsuffix='_last')
    
    # 予測した値を残して、ループ処理後に参照できるようにする
    pred_visitors_all = merge_data_trim.loc[:,["air_store_id","month","day", 'visitors','last_week']]

    # リークする危険があるのでか必ずtodayをdropして消す
    merge_data_trim = merge_data_trim.drop(columns='today')
    merge_data_trim = merge_data_trim.drop(columns='visitors')
    merge_data_trim = merge_data_trim.drop(columns='day_of_the_week_last')
    

    # test_fit_xを初期化する
    test_fit_x = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)

    #　テストデータのみ切り分けて、次の予測に用いる
    test_fit_x = merge_data_trim.loc[merge_data_trim.set == "test"]
    test_fit_x = test_fit_x.drop(columns='set')
    
print("done")

In [ ]:
# indexでソートする
pred = pred.sort_index()
pred

In [ ]:
pred = pred.reset_index(drop=True)
pred

In [ ]:
test_x_enc

In [ ]:
# pred_visitors_all.visitors
submission_ck = test_x_enc
submission_ck = submission_ck.join(pred.visitors)
submission_ck = submission_ck.join(pred.air_store_id, lsuffix='_pred')
submission_ck
# submission.at[252108,"visitors"]

In [ ]:
sns.scatterplot(x=submission_ck['air_store_id_pred'], y=submission_ck['air_store_id'])

In [ ]:
test

In [ ]:
submission = pd.DataFrame()
submission["id"] = test.id
submission["visitors"] = submission_ck.visitors

In [ ]:
submission

In [ ]:
# submission = pd.DataFrame({"id":test["id"], "visitors": pred})
# submission

# 提出用データの作成

In [ ]:
submission.to_csv("submission_13.csv", index=False)

# 提出用データのダウンロード
提出用データをダウンロードする<br>
データフレームの情報があれば、ダウンロードリンクを作成できる。

In [ ]:
# import the modules we'll need
from IPython.display import HTML
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "submission.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe
create_download_link(submission)

# 提出したデータを読み込む

In [ ]:
submit = pd.read_csv("/kaggle/working/submission_05.csv")

# 提出したデータと、予測に使ったデータを結合する